# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# assign the collection to a variable
establishments = db['establishments']

In [5]:
# Confirm that our new database was created
print("List of databases in MongoDB:")
print(mongo.list_database_names())

# Assign the uk_food database to a variable name
db = mongo['uk_food']

# Review the collections in our new database
print("Collections in the 'uk_food' database:")
print(db.list_collection_names())

# Review a document in the establishments collection
print("First document in the 'establishments' collection:")
pprint(db.establishments.find_one())

# Assign the collection to a variable
establishments = db['establishments']


List of databases in MongoDB:
['admin', 'config', 'local', 'met', 'uk_food']
Collections in the 'uk_food' database:
['establishments']
First document in the 'establishments' collection:
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('661d98ff5a2f0b59f342b84e'),
 'geocode

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [6]:
# Check that the new restaurant was inserted


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [7]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields


3. Update the new restaurant with the `BusinessTypeID` you found.

In [8]:
# Confirm that the new restaurant was updated


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [9]:
# Check that other documents remain with 'find_one'


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [10]:
# Change the data type from String to Decimal for longitude and latitude


Use `update_many` to convert `RatingValue` to integer numbers.

In [11]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [12]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

# Create an instance of MongoClient
mongo = MongoClient(port=27017)

# Assign the uk_food database to a variable name
db = mongo['uk_food']

# Assign the establishments collection to a variable
establishments = db['establishments']

# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "LocalAuthorityName": "Greenwich",
    "RatingValue": None  # No rating yet
}

# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)
print("New restaurant inserted with ID:", insert_result.inserted_id)

# Check that the new restaurant was inserted
print("Newly inserted restaurant:")
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))

# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0})

# Check if the query returned a result
if business_type:
    print("Business Type ID for Restaurant/Cafe/Canteen:", business_type)
    
    # Update the new restaurant with the correct BusinessTypeID
    update_result = establishments.update_one({"BusinessName": "Penang Flavours"}, {"$set": {"BusinessTypeID": business_type["BusinessTypeID"]}})
    
    # Check if the update was successful
    if update_result.modified_count > 0:
        print("New restaurant updated with BusinessTypeID:", business_type["BusinessTypeID"])
    else:
        print("No updates made to the new restaurant.")
else:
    print("Business Type ID for Restaurant/Cafe/Canteen not found.")

# Find how many documents have LocalAuthorityName as "Dover"
num_dover_establishments = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Number of establishments in Dover:", num_dover_establishments)

# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print("Number of documents deleted:", delete_result.deleted_count)

# Check if any remaining documents include Dover
remaining_dover_establishments = establishments.find_one({"LocalAuthorityName": "Dover"})
print("Remaining documents with Dover:", remaining_dover_establishments)

# Check that other documents remain with 'find_one'
print("Example remaining document:")
pprint(establishments.find_one())

# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, {'$set': {"RatingValue": None}})

# Check that the coordinates and rating value are now numbers
example_document = establishments.find_one()
print("Example document with converted values:")
pprint(example_document)


New restaurant inserted with ID: 6630631842a5a2765aeb4a76
Newly inserted restaurant:
{'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 'LocalAuthorityName': 'Greenwich',
 'RatingValue': None,
 '_id': ObjectId('661d998a32ea33e007202a47')}
Business Type ID for Restaurant/Cafe/Canteen: {'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
No updates made to the new restaurant.
Number of establishments in Dover: 0
Number of documents deleted: 0
Remaining documents with Dover: None
Example remaining document:
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName':